In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import json

from agent.mcts import ChessMonteCarloTreeSearch
from agent.predict import MovePredictor
from chess_environment.position_parser import ChessPositionParser

import config

np.random.seed(123)

cfg = config.Config()

Using Theano backend.


In [2]:
# Load ECO file of openings

with open(cfg.resources.eco_file) as f:
    ecos = json.load(f)

ecos


[{'c': 'A00',
  'f': 'rn1qkbnr/ppp2ppp/8/3p4/5p2/6PB/PPPPP2P/RNBQK2R w KQkq',
  'id': 2990,
  'm': 'g1h3 d7d5 g2g3 e7e5 f2f4 c8h3 f1h3 e5f4',
  'n': 'Amar Gambit'},
 {'c': 'A00',
  'f': 'rn1qkbnr/ppp2ppp/8/3p4/8/6PB/PPPPP3/RNBQ1RK1 b kq',
  'id': 46,
  'm': 'g1h3 d7d5 g2g3 e7e5 f2f4 c8h3 f1h3 e5f4 e1g1 f4g3 h2g3',
  'n': 'Amar Opening: Gent Gambit'},
 {'c': 'A00',
  'f': 'rnbqkbnr/ppp2ppp/8/3pp3/5P2/6PN/PPPPP2P/RNBQKB1R b KQkq',
  'id': 47,
  'm': 'g1h3 d7d5 g2g3 e7e5 f2f4',
  'n': 'Amar Opening: Paris Gambit'},
 {'c': 'A00',
  'f': 'r1bqkb1r/ppp2ppp/2np1n2/4p3/2P5/1PN1P3/P2P1PPP/R1BQKBNR w KQkq',
  'id': 2994,
  'm': 'e2e3 e7e5 c2c4 d7d6 b1c3 b8c6 b2b3 g8f6',
  'n': 'Amsterdam Attack'},
 {'c': 'A00',
  'f': 'rnbqkbnr/pppppppp/8/8/8/P7/1PPPPPPP/RNBQKBNR b KQkq',
  'id': 49,
  'm': 'a2a3',
  'n': 'Anderssen Opening'},
 {'c': 'A00',
  'f': 'rnbqkbnr/1ppppppp/8/p7/1P6/P7/2PPPPPP/RNBQKBNR b KQkq',
  'id': 50,
  'm': 'a2a3 a7a5 b2b4',
  'n': 'Anderssen Opening: Polish Gambit'},
 {'c': 'A00'

In [3]:
# Load up the trained model
move_predictor = MovePredictor(cfg)
position_parser = ChessPositionParser(cfg, [], [])

elos = [1500]
time_control = '3600+30'
num_simulations = 25

results = []

mcts = ChessMonteCarloTreeSearch(cfg, move_predictor, position_parser, num_simulations=num_simulations)

def save_results(results):
    j = json.dumps(results, indent=2)

    with open('/Users/tom/Projects/Portfolio/web-static/public/chess/openings-strength.json', 'w') as f:
        print(j, file=f)

count = 0
total = len(ecos) * len(elos)

try:
    for opening in ecos:
        for elo in elos:
            fen = opening['f'] + ' - 0 1'
            ab_eval, mcts_eval = mcts.set_position(fen, player_elo=elo, opponent_elo=elo).run_search().get_evals()
            results.append({
                'eco': opening['c'],
                'fen': opening['f'],
                'moves': opening['m'],
                'name': opening['n'],
                'player_elo': elo,
                'ab_eval': ab_eval,
                'mcts_eval': mcts_eval,
            })

            count += 1
            print(f'{count} of {total}')
            
except KeyboardInterrupt:
    save_results(results)
    raise KeyboardInterrupt
        
save_results(results)
        





1 of 3414
2 of 3414
3 of 3414
4 of 3414
5 of 3414
6 of 3414
7 of 3414
8 of 3414
9 of 3414
10 of 3414
11 of 3414
12 of 3414
13 of 3414
14 of 3414
15 of 3414
16 of 3414
17 of 3414
18 of 3414
19 of 3414
20 of 3414
21 of 3414
22 of 3414
23 of 3414
24 of 3414
25 of 3414
26 of 3414
27 of 3414
28 of 3414
29 of 3414
30 of 3414
31 of 3414
32 of 3414
33 of 3414
34 of 3414
35 of 3414
36 of 3414
37 of 3414
38 of 3414
39 of 3414
40 of 3414
41 of 3414
42 of 3414
43 of 3414
44 of 3414
45 of 3414
46 of 3414
47 of 3414
48 of 3414
49 of 3414
50 of 3414
51 of 3414
52 of 3414
53 of 3414
54 of 3414
55 of 3414
56 of 3414
57 of 3414
58 of 3414
59 of 3414
60 of 3414
61 of 3414
62 of 3414
63 of 3414
64 of 3414
65 of 3414
66 of 3414
67 of 3414
68 of 3414
69 of 3414
70 of 3414
71 of 3414
72 of 3414
73 of 3414
74 of 3414
75 of 3414
76 of 3414
77 of 3414
78 of 3414
79 of 3414
80 of 3414
81 of 3414
82 of 3414
83 of 3414
84 of 3414
85 of 3414
86 of 3414
87 of 3414
88 of 3414
89 of 3414
90 of 3414
91 of 3414
92 of 34

In [4]:
import pandas as pd

df = pd.DataFrame(results)
df['diff_eval'] = df['ab_eval'] - df['mcts_eval']
df['abs_diff_eval'] = df['diff_eval'].abs()

df = df.sort_values(by='abs_diff_eval', ascending=False)

df.head()

,ab_eval,eco,fen,mcts_eval,moves,name,player_elo,diff_eval,abs_diff_eval
1883,0.969745,C37,rnb2bnr/pppp1k1p/5q2/8/4P3/2N1pQ2/PPP3PP/R4RK1...,-0.284953,e2e4 e7e5 f2f4 e5f4 g1f3 g7g5 f1c4 g5g4 e1g1 g...,"King's Gambit Accepted, Double Muzio Gambit, Y...",1500,1.254698,1.254698
2125,-0.201216,C44,r1bqkb1r/pp1p1Npp/2p1nn2/8/2B1P3/8/PPP2PPP/RNB...,0.830207,e2e4 e7e5 g1f3 b8c6 d2d4 c6d4 f3e5 d4e6 f1c4 c...,"Scotch, Cochrane Variation",1500,-1.031424,1.031424
2388,-0.206966,C60,r1b1kbnr/ppp2ppp/2N5/1B1p2q1/4P3/8/PPPP1PPP/RN...,0.725584,e2e4 e7e5 g1f3 b8c6 f1b5 d7d5 f3e5 d8g5 e5c6,"Ruy Lopez: Spanish Countergambit, Harding Gambit",1500,-0.932550,0.932550
2359,0.212702,C58,r1bqkb1r/p4ppp/5n2/np2p1N1/8/5Q2/PPPP1PPP/RNB1...,-0.631178,e2e4 e7e5 g1f3 b8c6 f1c4 g8f6 f3g5 d7d5 e4d5 c...,"Italian Game: Two Knights Defense, Blackburne ...",1500,0.843881,0.843881
1785,0.543423,C30,rnb1k1nr/pppp1ppp/8/4p3/1bB1Pq2/2N2N2/PPPP2PP/...,-0.272510,e2e4 e7e5 f2f4 d8f6 b1c3 f6f4 g1f3 f8b4 f1c4,"King's Gambit Declined: Norwalde Variation, Bu...",1500,0.815932,0.815932


In [5]:
df

,ab_eval,eco,fen,mcts_eval,moves,name,player_elo,diff_eval,abs_diff_eval
1883,0.969745,C37,rnb2bnr/pppp1k1p/5q2/8/4P3/2N1pQ2/PPP3PP/R4RK1...,-0.284953,e2e4 e7e5 f2f4 e5f4 g1f3 g7g5 f1c4 g5g4 e1g1 g...,"King's Gambit Accepted, Double Muzio Gambit, Y...",1500,1.254698,1.254698
2125,-0.201216,C44,r1bqkb1r/pp1p1Npp/2p1nn2/8/2B1P3/8/PPP2PPP/RNB...,0.830207,e2e4 e7e5 g1f3 b8c6 d2d4 c6d4 f3e5 d4e6 f1c4 c...,"Scotch, Cochrane Variation",1500,-1.031424,1.031424
2388,-0.206966,C60,r1b1kbnr/ppp2ppp/2N5/1B1p2q1/4P3/8/PPPP1PPP/RN...,0.725584,e2e4 e7e5 g1f3 b8c6 f1b5 d7d5 f3e5 d8g5 e5c6,"Ruy Lopez: Spanish Countergambit, Harding Gambit",1500,-0.932550,0.932550
2359,0.212702,C58,r1bqkb1r/p4ppp/5n2/np2p1N1/8/5Q2/PPPP1PPP/RNB1...,-0.631178,e2e4 e7e5 g1f3 b8c6 f1c4 g8f6 f3g5 d7d5 e4d5 c...,"Italian Game: Two Knights Defense, Blackburne ...",1500,0.843881,0.843881
1785,0.543423,C30,rnb1k1nr/pppp1ppp/8/4p3/1bB1Pq2/2N2N2/PPPP2PP/...,-0.272510,e2e4 e7e5 f2f4 d8f6 b1c3 f6f4 g1f3 f8b4 f1c4,"King's Gambit Declined: Norwalde Variation, Bu...",1500,0.815932,0.815932
2132,-0.710404,C45,N1bk3r/pp1p1ppp/2n2n2/8/1b6/P4B1q/1PPN1P1P/R1B...,0.029961,e2e4 e7e5 g1f3 b8c6 d2d4 e5d4 f3d4 d8h4 d4b5 f...,Scotch Game: Berger Variation,1500,-0.740365,0.740365
2244,-0.613179,C51,r2q1knr/pppb1Bpp/1b1p4/n7/3PP3/2N2N2/P1Q2PPP/R...,0.114570,e2e4 e7e5 g1f3 b8c6 f1c4 f8c5 b2b4 c5b4 c2c3 b...,"Italian Game: Evans Gambit, Fraser-Mortimer At...",1500,-0.727749,0.727749
1638,0.384219,C21,rnbqkbnr/p2p1ppp/8/1pp5/2BpP3/5N2/PPP2PPP/RNBQ...,-0.270762,e2e4 e7e5 d2d4 e5d4 g1f3 c7c5 f1c4 b7b5,Center Game: Kieseritzky Variation,1500,0.654981,0.654981
2937,-0.047963,D44,rnbqkb1r/p4p2/2p1pn1p/1p2P1N1/2pP3B/2N5/PP3PPP...,0.591256,d2d4 d7d5 c2c4 c7c6 g1f3 g8f6 b1c3 e7e6 c1g5 d...,Semi-Slav Defense: Botvinnik System,1500,-0.639219,0.639219
2011,-0.315823,C41,r1bqk2r/pppnbBpp/3p1n2/4p3/3PP3/2N2N2/PPP2PPP/...,0.262523,e2e4 e7e5 g1f3 d7d6 d2d4 g8f6 b1c3 b8d7 f1c4 f...,"Philidor Defense: Lion Variation, Bishop Sacri...",1500,-0.578346,0.578346


In [9]:
import chess
import math

def map_san(moves):
    board = chess.Board()
    moves_san = ''
    for idx, m in enumerate(moves.split(' ')):
        half_move = idx + 1
        
        move = chess.Move.from_uci(m)
        move_san = board.san(move)
        
        if half_move % 2:
            moves_san += '{}. '.format(math.ceil(half_move / 2))
            
        moves_san += '{} '.format(move_san)
        
        board.push(move)
        
    return moves_san

df['moves_san'] = df['moves'].map(map_san)

df.to_pickle('/Users/tom/Projects/Portfolio/data/chess-deep-learning/openings.p')